In [30]:
Datadir = './data/'

import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import inspect  

one_one_size_small=[5,4]
one_two_size_small=[7,4]
one_three_size_large=[10,4]
one_four_size_large=[12,4]
one_five_size_large=[14,4]

two_one_size_small=[5,6]
two_two_size_small=[7,6]
two_four_size_large=[11,6]
two_two_size_large=[7,6]

three_one_size_small=[5,8]
three_two_size_small=[7,8]

Hubble_h = 0.67

def read_file(fa):
    fa = open(fa, "r") 
    index=0
    for line in fa:
        if(index==0):                
            fields = line.strip().split()  
            N_elements=int(fields[0])
            x_axis=np.zeros(N_elements,dtype=np.float32)
            y_axis=np.zeros(N_elements,dtype=np.float32)               
        else:
            fields = line.strip().split()               
            x_axis[index-1]=float(fields[0])
            y_axis[index-1]=float(fields[1])               
        index+=1    
        
    return(x_axis,y_axis)  

#end read_file

def plot_label (subplot, label_type, xlim, ylim, x_percentage, y_percentage, color='white', 
                x2_percentage=0., xlog=0, ylog=0, label='', linestyle='-', linewidth=2, 
                fontsize=16, fontweight='normal', sym='o', sym_size=5, err_size=0.1, 
                rotation=0,backgroundcolor='none', alpha=1.,back_alpha=0.,mfc='black', 
                facecolors='none', edgecolors='white'):    
    
    if(mfc=='black'):
        mfc=color
    
    if xlog==0:
        x = xlim[0]+(xlim[1]-xlim[0])*x_percentage
        x2 = xlim[0]+(xlim[1]-xlim[0])*x2_percentage
    else:     
        if(xlim[0]==0):
            x_low=0.
        else:
            x_low=np.log10(xlim[0])
        x = 10**(x_low+(np.log10(xlim[1])-x_low)*x_percentage/10.)       
        x2 = 10**(x_low+(np.log10(xlim[1])-x_low)*x2_percentage/10.)
        #print(x)      
    if ylog==0:
        y = ylim[0]+(ylim[1]-ylim[0])*y_percentage
    else:    
        if(ylim[0]==0):
            y_low=0.
        else:
            y_low=np.log10(ylim[0])
        y = 10**(y_low+(np.log10(ylim[1])-y_low)*y_percentage/10.)
        
    if label_type=='label':                   
        t=subplot.text(x,y,label, fontsize=fontsize, fontweight=fontweight,rotation=rotation, 
                       color=color, backgroundcolor=backgroundcolor, alpha=alpha)
        t.set_bbox(dict(alpha=back_alpha, color=backgroundcolor))
    else:
        if label_type =='line':              
            subplot.plot([x,x2],[y,y],color=color,linestyle=linestyle, linewidth=linewidth, alpha=alpha)
                
        else:
            if label_type=='symbol':                  
                if(err_size>0.):
                    subplot.errorbar(x,y, yerr=err_size, fmt=sym, markersize=sym_size, 
                                     color=color, markeredgecolor=color,mfc=mfc)
                else:
                    subplot.scatter(x,y, marker=sym, s=sym_size, facecolors=facecolors, edgecolors=edgecolors)
                     
#end plot_label

def stellar_mass_vs_halo_mass_fractional():

    xlim=[11., 14.5]
    ylim=[-3.,0.]    
      
    #All models
    fig = plt.figure(figsize=(one_four_size_large[0],one_four_size_large[1]))
    grid = gridspec.GridSpec(1, 4)
    grid.update(wspace=0.0, hspace=0.0)

    
    model_names=['Hen15_no_feedback','Hen15_only_AGN','Hen15_only_SN','Hen15']
    #model_names=['Hen15_other','Hen15_only_AGN','Hen15_only_SN','Hen15']
    model_label=['no feedback','only AGN','only SN','Hen15']
    
    for ii in range(0,4):
        
        subplot=plt.subplot(grid[ii])

        subplot.set_ylim(ylim), subplot.set_xlim(xlim)
        xlab='$\log_{10}(M_{\mathrm{200c}}[\mathrm{M}_{\odot}])$' 
        subplot.set_xlabel(xlab, fontsize=14 ,fontweight='bold')
      
        if ii==0:
            ylab='$\log_{10}(M_*/M_{\mathrm{200c}})$'
            subplot.set_ylabel(ylab, fontsize=14)
                       
    
        majorFormatter = FormatStrFormatter('%d')
        subplot.xaxis.set_major_locator(MultipleLocator(1))    
        subplot.xaxis.set_minor_locator(MultipleLocator(0.25))      
        subplot.yaxis.set_minor_locator(MultipleLocator(0.25))
  
        if ii>0:
            plt.tick_params(axis='y', which='both', left='on', labelleft='off')
      
        #Behroozi2013
        z=0.0 
        a=1/(1+z)
        neu=np.exp(-4*a*a)
        log_epsilon=-1.777+(-0.006*(a-1)-0.*z)*neu-0.119*(a-1)
        log_M1=11.514+(-1.793*(a-1)-0.251*z)*neu
        alpha=-1.412+0.731*(a-1.)*neu
        delta=3.508+(2.608*(a-1)-0.043*z)*neu
        gamma=0.316+(1.319*(a-1)+0.279*z)*neu 
               
        x=0.
        f_0=-np.log10(10**(alpha*x) + 1) + delta * ((np.log10(1 + np.exp(x)))**gamma)/(1 + np.exp(10**(-x)))             
        log_Mh=np.arange(8.0,16.0,0.01) 
        x=log_Mh-log_M1
        f_log_Mh=-np.log10(10**(alpha*x) + 1) + delta * ((np.log10(1 + np.exp(x)))**gamma)/(1 + np.exp(10**(-x)))      
        log_mstar=log_epsilon+log_M1+f_log_Mh-f_0     
        subplot.plot(log_Mh, log_mstar-log_Mh,color='limegreen', linewidth=3)  
        
        #MODELS       
        fa = Datadir+"SMHM_"+model_names[ii]+"_SF_z0.0.txt"
        (x_axis,y_axis)=read_file(fa)    
        subplot.scatter(x_axis,y_axis,s=5, color='blue') 
        
        fa = Datadir+"SMHM_"+model_names[ii]+"_passive_z0.0.txt"
        (x_axis,y_axis)=read_file(fa)    
        subplot.scatter(x_axis,y_axis,s=5, alpha=0.8, color='red') 
            
        fa = Datadir+"SMHM_"+model_names[ii]+"_median_z0.0.txt"
        (x_axis,y_axis)=read_file(fa)
        subplot.plot(x_axis, y_axis,color='black', linewidth=2)   
         
        fa = Datadir+"SMHM_"+model_names[ii]+"_pc16_z0.0.txt"
        (x_axis,y_axis)=read_file(fa)
        subplot.plot(x_axis, y_axis,color='black', linewidth=2, linestyle='--') 
         
        fa = Datadir+"SMHM_"+model_names[ii]+"_pc84_z0.0.txt"
        (x_axis,y_axis)=read_file(fa)
        subplot.plot(x_axis, y_axis,color='black', linewidth=2, linestyle='--')  
    
        x_arr=np.arange(xlim[0],xlim[1]+0.05,0.05)           
        y_arr=x_arr*0.+np.log10(0.155)      
        subplot.plot(x_arr,y_arr,color='black', linestyle='--', linewidth=3)  
        
        fa = Datadir+"SMHM_"+model_names[ii]+"_median_with_sats_z0.0.txt"
        (x_axis,y_axis)=read_file(fa)       
        subplot.plot(x_axis,y_axis,color='black', linewidth=2, linestyle=':') 
         
        if(ii==0):
            plot_label (subplot, 'label', xlim, ylim, x_percentage=0.15, y_percentage=0.24, 
                        color='black', xlog=0, ylog=0, label='Model Median', fontsize=10, fontweight='normal') 
            plot_label (subplot, 'line', xlim, ylim, x_percentage=0.06, y_percentage=0.26, x2_percentage=0.12, 
                        color='black', xlog=0, ylog=0, linestyle='-', linewidth=2)  
            
            plot_label (subplot, 'label', xlim, ylim, x_percentage=0.15, y_percentage=0.17, 
                        color='black', xlog=0, ylog=0, label='Centrals + sats + ICL', fontsize=10, fontweight='normal') 
            plot_label (subplot, 'line', xlim, ylim, x_percentage=0.06, y_percentage=0.19, x2_percentage=0.12, 
                        color='black', xlog=0, ylog=0, linestyle=':', linewidth=2)  
            
            plot_label (subplot, 'label', xlim, ylim, x_percentage=0.15, y_percentage=0.10, 
                        color='black', xlog=0, ylog=0, label='Behroozi et al. (2013)', fontsize=10, fontweight='normal') 
            plot_label (subplot, 'line', xlim, ylim, x_percentage=0.06, y_percentage=0.12, x2_percentage=0.12, 
                        color='limegreen', xlog=0, ylog=0, linestyle='-', linewidth=2) 
        
        plot_label (subplot, 'label', xlim, ylim, x_percentage=0.88, y_percentage=0.76,               
                    color='black', xlog=0, ylog=0, label='$f_b$', 
                    fontsize=15, fontweight='normal')
        
        plot_label (subplot, 'label', xlim, ylim, x_percentage=0.1, y_percentage=0.85, 
                    color='black', xlog=0, ylog=0, label=model_label[ii], 
                    fontsize=12, fontweight='normal')
        
    #endfor
    
    
    plt.tight_layout()
    current_function =  inspect.getframeinfo(inspect.currentframe()).function   
    plt.savefig('./fig/plots_'+current_function+'.pdf')
    plt.close()
    
    return 

print("done")

done


In [31]:
stellar_mass_vs_halo_mass_fractional()
print('done')

/Users/BrunoHenriques/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:142: RuntimeWarning: overflow encountered in exp
/Users/BrunoHenriques/anaconda/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


done
